In [1]:
from alpha_vantage.fundamentaldata import FundamentalData
import pandas as pd
import time
import sqlite3

In [2]:
# apikey = '4YLKM5SFV5RXMQCG'
# fd = FundamentalData(apikey, output_format = 'pandas')

# symbol = 'ACAD'

# data = fd.get_balance_sheet_quarterly(symbol)
# data1 = fd.get_income_statement_quarterly(symbol)
# data = data[0].T
# data1 = data1[0].T

In [3]:
# result = pd.concat([data, data1])
# result = result.T.reset_index()
# new_result = result.loc[:,~result.columns.duplicated(keep='first')]
# new_result = new_result.drop(new_result.columns[[0,2]], axis=1)

# new_result['netIncome'] = result['netIncome']
# new_result.iloc[:, 1:] = new_result.iloc[:, 1:].apply(lambda x: pd.to_numeric(x, errors='coerce')).astype(float)

# Equity = (new_result['totalAssets'] - new_result['totalLiabilities'])
# NetProfit = (new_result['grossProfit'] - new_result['operatingExpenses'])
# ROA_value = (new_result['netIncome'] / new_result['totalAssets']) * 100
# ROE_value = (new_result['netIncome'] / new_result['totalShareholderEquity']) * 100

# new_result['Equity'] = Equity
# new_result['NetProfit'] = NetProfit
# new_result['ROA'] = ROA_value
# new_result['ROE'] = ROE_value
# new_result['Symbol'] = symbol

In [4]:
# result_after = new_result.drop(new_result.columns[[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,20,21,22,23,24,25,26,27,28,29,30,
#                                           31,32,33,34,35,36,37,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60]]
#                                           , axis=1)
# result_after = result_after.round(3)
# result_after = result_after.rename(columns={"fiscalDateEnding": "Period", "totalAssets": "Asset", "totalLiabilities": "TotalDebt",
#                              "totalRevenue": "Revenue"})
# result_after = result_after[['Asset', 'TotalDebt', 'Equity', 'Revenue', 'NetProfit', 'ROA', 'ROE', 'Period', 'Symbol']]
# result_after['Period'] = pd.to_datetime(result_after['Period'])
# result_after = result_after[~(result_after['Period'].isna())]
# result_after = result_after.sort_values(by=['Period'], ascending=True).reset_index(drop=True)
# result_after['Period'] = result_after['Period'].dt.quarter.astype(str) +'Q'+ result_after['Period'].dt.year.astype(str)

# result_after

In [5]:
# result_after.dtypes

In [6]:
def get_symbol_id(market):
        con = sqlite3.connect('share_V2.sqlite')
        cur = con.cursor()
        sql = f"""select Information.Symbol,Information.SymbolId  
                from Information Inner join Market on Market.MarketId = Information.MarketId
                where Market.Mname = "{market}"
                """
        cur.execute(sql)
        records = cur.fetchall()
        con.close()
        symbol_id = {}
        for share in records:
            symbol_id[share[0]] = share[1]
        return symbol_id

In [7]:
def alphaVantage_data(symbol):
    apikey = '4YLKM5SFV5RXMQCG'
    fd = FundamentalData(apikey, output_format = 'pandas')
    balance_sheet = fd.get_balance_sheet_quarterly(symbol)
    income_state = fd.get_income_statement_quarterly(symbol)
    balance_sheet = balance_sheet[0].T
    income_state = income_state[0].T
    return (balance_sheet, income_state)

def calInfo(data):
    Equity = (data['totalAssets'] - data['totalLiabilities'])
    NetProfit = (data['grossProfit'] - data['operatingExpenses'])
    ROA_value = (data['netIncome'] / data['totalAssets']) * 100
    ROE_value = (data['netIncome'] / data['totalShareholderEquity']) * 100
    data['Equity'] = Equity
    data['NetProfit'] = NetProfit
    data['ROA'] = ROA_value
    data['ROE'] = ROE_value
    return data

def arrangeAlphaVantageDetail(symbol):
    data_balance, data_income = alphaVantage_data(symbol)
    result = pd.concat([data_balance, data_income])
    result = result.T.reset_index()
    new_result = result.loc[:,~result.columns.duplicated(keep='first')]
    new_result = new_result.drop(new_result.columns[[0,2]], axis=1)
    new_result['netIncome'] = result['netIncome']
    new_result.iloc[:, 1:] = new_result.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
    new_result = calInfo(new_result)
    new_result['SymbolId'] = str(int(get_symbol_id("NASDAQ")[symbol]))
    result_after = new_result.drop(new_result.columns[[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,20,21,22,23,24,25,26,27,28,29,30,
                                          31,32,33,34,35,36,37,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60]]
                                          , axis=1)
    result_after = result_after.round(3)
    result_after = result_after.rename(columns={"fiscalDateEnding": "Period", "totalAssets": "Asset", "totalLiabilities": "TotalDebt",
                                "totalRevenue": "Revenue"})
    result_after = result_after[['Asset', 'TotalDebt', 'Equity', 'Revenue', 'NetProfit', 'ROA', 'ROE', 'Period', 'SymbolId']]
    result_after['Period'] = pd.to_datetime(result_after['Period'])
    result_after = result_after[~(result_after['Period'].isna())]
    result_after = result_after.sort_values(by=['Period'], ascending=True).reset_index(drop=True)
    result_after['Period'] = result_after['Period'].dt.quarter.astype(str) +'Q'+ result_after['Period'].dt.year.astype(str)
    return result_after

In [8]:
result = arrangeAlphaVantageDetail('META')
result

NameError: name 'get_symbol_id' is not defined

In [ ]:
# def AlphaVantageDetail():
#     values = allSymbolnasdaq()[:50]
#     details_alpha = []
#     count = 0
#     for name in values:
#         if (count == 2):
#             time.sleep(65)
#             df = arrangeAlphaVantageDetail(name)
#             df_old = details_alpha[0]
#             df_new = pd.concat([df_old, df])
#             details_alpha[0] = df_new
#             count = 0
#         elif len(details_alpha) == 0:
#             df1 = arrangeAlphaVantageDetail(name)
#             details_alpha.append(df1)
#         else:
#             df2 = arrangeAlphaVantageDetail(name)
#             df_old = details_alpha[0]
#             df_new = pd.concat([df_old, df2])
#             details_alpha[0] = df_new
#         count+=1
#     return details_alpha[0]